# Dataset
https://www.kaggle.com/mrisdal/fake-news#fake.csv

Example domain classifications (in flux) include:

  * **Fake News**: Sources that fabricate stories out of whole cloth with the intent of pranking the public.
  * **Satire**: Sources that provide humorous commentary on current events in the form of fake news.
  *  **Extreme Bias**: Sources that traffic in political propaganda and gross distortions of fact.
  *  **Conspiracy Theory**: Sources that are well-known promoters of kooky conspiracy theories.
  *  **Rumor Mill**: Sources that traffic in rumors, innuendo, and unverified claims.
  *  **State News**: Sources in repressive states operating under government sanction.
  *  **Junk Science**: Sources that promote pseudoscience, metaphysics, naturalistic fallacies, and other scientifically dubious claims.
  *  **Hate Group**: Sources that actively promote racism, misogyny, homophobia, and other forms of discrimination.
  *  **Clickbait:** Sources that are aimed at generating online advertising revenue and rely on sensationalist headlines or eye-catching pictures.
  *  **Proceed With Caution**: Sources that may be reliable but whose contents require further verification.
  *  The labels of **bs** and **junksci**, etc. do not constitute capital "t" Truth



In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
#import mlutils
%matplotlib inline
import pandas as pd
import tensorflow_hub as hub 
import re
print(tf.__version__)
print("The Python version is %s.%s.%s" % sys.version_info[:3])


1.10.1
The Python version is 2.7.14


## Descarga, arregla e inspecciona el dataset Fake news

In [0]:
import os
!mkdir -p dataFake
if not os.path.exists("dataFake/fake.csv"):
    !wget -c -nc https://github.com/chdorrego/tf/raw/master/fake-news.zip -P dataFake
    !unzip dataFake/fake-news.zip -d dataFake
    with open("dataFake/fake.csv", "r") as f:
        lines = f.readlines()
        lines = [i for i in lines if len(i.replace("\n", ""))>0]
    with open("dataFake/fake.csv", "w") as f:
        f.writelines(lines) 

In [3]:
!head -5 dataFake/fake.csv

"uuid","ord_in_thread","author","published","title","text","language","crawled","site_url","country","domain_rank","thread_title","spam_score","main_img_url","replies_count","participants_count","likes","comments","shares","type"
"6a175f46bcd24d39b3e962ad0f29936721db70db",0,"Barracuda Brigade","2016-10-26T21:41:00.000+03:00","Muslims BUSTED: They Stole Millions In Gov’t Benefits","Print They should pay all the back all the money plus interest. The entire family and everyone who came in with them need to be deported asap. Why did it take two years to bust them? 
Here we go again …another group stealing from the government and taxpayers! A group of Somalis stole over four million in government benefits over just 10 months! 
We’ve reported on numerous cases like this one where the Muslim refugees/immigrants commit fraud by scamming our system…It’s way out of control! More Related","english","2016-10-27T01:49:27.168+03:00","100percentfedup.com","US",25689,"Muslims BUSTED: They Stole Millio

In [3]:
CSV_COLUMNS  = ['uuid','ord_in_thread','author','published','title','text','language','crawled','site_url','country','domain_rank','thread_title','spam_score','main_img_url','replies_count','participants_count','likes','comments','shares','type_label']
#CSV_DEFAULTS = [[''],[0],[''],[''],[''],[''],[''],[''],[''],[''],[0],[''],[-1],[''],[-1],[-1],[-1],[-1],[-1],['']]
CSV_DEFAULTS = [[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[0]]


type_label =  ['bias','conspiracy','hate','state','junksci','satire','fake']
no_valid_labels = ['bs']

fake= pd.read_csv("dataFake/fake.csv", names = CSV_COLUMNS,skiprows=1)
print(fake.shape)
fake.head()

(12999, 20)


,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type_label
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


### Labels

* bs            10919 
* conspiracy      412 
- bias            354 
- hate            246 
- state           121 
- junksci         102 
- satire          100 
- fake             19


## Realizamos un preprocesamiento de los datos 
- Eliminiación de la clase bs debido a la definición del problema
- Se eliminan los caracteres especiales
- Se transforma el valor del target por una equivalencia númerica para facilidad del procesamiento
- Realizamos la división de la data en archivos test y train 


In [4]:
d = fake.loc[fake['type_label'].isin(type_label)]
d['title'] = d['title'].str.replace('\W', ' ')
d['text'] =  d['text'].str.replace('\W', ' ')
d['thread_title'] =  d['thread_title'].str.replace('\W', '')

d['type_label'] = d['type_label'] 
d['type_label'] = d['type_label'].astype("category", categories=type_label).cat.codes


msk = np.random.rand(len(d)) < 0.8

d_train = d[msk==True]
d_test =  d[msk==False]

d_train.to_csv('dataFake/fake_train.csv', sep=';', encoding='utf-8', header=False)
d_test.to_csv('dataFake/fake_test.csv', sep=';', encoding='utf-8', header=False)
print(d_train.count())
print(d_test.count())

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

uuid                  1191
ord_in_thread         1191
author                 813
published             1191
title                 1100
text                  1152
language              1191
crawled               1191
site_url              1191
country               1191
domain_rank           1046
thread_title          1191
spam_score            1191
main_img_url           931
replies_count         1191
participants_count    1191
likes                 1191
comments              1191
shares                1191
type_label            1191
dtype: int64
uuid                  316
ord_in_thread         316
author                215
published             316
title                 300
text                  309
language              316
crawled               316
site_url              316
country               316
domain_rank           273
thread_title          316
spam_score            316
main_img_url          262
replies_count         316
participants_count    316
likes                 316
comme

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


## Creamos la funcion para la generacion del `input pipeline`

In [0]:
CSV_COLUMNS  = ['pd_rowid','uuid','ord_in_thread','author','published','title','text','language','crawled','site_url','country','domain_rank','thread_title','spam_score','main_img_url','replies_count','participants_count','likes','comments','shares','type_label']
CSV_DEFAULTS = [[0],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[0]]


def parse_csv(rows_string_tensor, use_features):
    columns = tf.decode_csv(rows_string_tensor, record_defaults=CSV_DEFAULTS, field_delim=";")
    features = dict(zip(CSV_COLUMNS, columns))   
    
    label = features.pop("type_label")
      
    #label = tf.one_hot(indices=label,depth=len(type_label),dtype=tf.int32)    
    
    return {k:v for k,v in features.items() if k in use_features}, label



def get_dataset(filename, repeat, batch_size, shuffle):
    use_features  = ['title','type_label']
    
    dataset = tf.data.TextLineDataset(filename)
    dataset = dataset.repeat(repeat)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=300)
    dataset = dataset.map(lambda x: parse_csv(x, use_features))    
    dataset = dataset.batch(batch_size)
    return dataset



def get_features_defs():
  
    title = hub.text_embedding_column(
        key='title', 
        module_spec='https://tfhub.dev/google/Wiki-words-500/1')
    
    
    return [title] 
  
  
#Este metodo no se usa en el premade - solo se utiliza para realiza una prueba
def make_data_iterator(filename, repeat, batch_size, shuffle=True):
    
    dataset = get_dataset(filename, repeat, batch_size, shuffle)
    
    features_defs = get_features_defs()
    
    features, label = dataset.make_one_shot_iterator().get_next()
    
    next_X = tf.feature_column.input_layer(features, features_defs)
    next_y = label
    
    return next_X, next_y

Prueba de la carga del dataset 

In [7]:
dataset = tf.data.TextLineDataset("dataFake/fake_train.csv")
dataset = dataset.batch(10)
features = dataset.make_one_shot_iterator().get_next()
with tf.Session() as sess:
    print(sess.run(features))


['0;6a175f46bcd24d39b3e962ad0f29936721db70db;0;Barracuda Brigade;2016-10-26T21:41:00.000+03:00;Muslims BUSTED  They Stole Millions In Gov   t Benefits;Print They should pay all the back all the money plus interest  The entire family and everyone who came in with them need to be deported asap  Why did it take two years to bust them   Here we go again    another group stealing from the government and taxpayers  A group of Somalis stole over four million in government benefits over just 10 months   We   ve reported on numerous cases like this one where the Muslim refugees immigrants commit fraud by scamming our system   It   s way out of control  More Related;english;2016-10-27T01:49:27.168+03:00;100percentfedup.com;US;25689.0;MuslimsBUSTEDTheyStoleMillionsInGovtBenefits;0.0;http://bb4sp.com/wp-content/uploads/2016/10/Fullscreen-capture-10262016-83501-AM.bmp.jpg;0;1;0;0;0;0'
 '2;c70e149fdd53de5e61c29281100b9de0ed268bc3;0;Barracuda Brigade;2016-10-31T01:41:49.479+02:00;BREAKING  Weiner Coo

Realizamos una preuba sobre el metodo parse_csv

In [8]:
dataset = tf.data.TextLineDataset("dataFake/fake_train.csv")
dataset = dataset.batch(2)
use_features  = ['title','type_label']
dataset = dataset.map(lambda x: parse_csv(x, use_features))     
features = dataset.make_one_shot_iterator().get_next()
with tf.Session() as sess:
    print(sess.run(features))

({'title': array(['Muslims BUSTED  They Stole Millions In Gov   t Benefits',
       'BREAKING  Weiner Cooperating With FBI On Hillary Email Investigation'],
      dtype=object)}, array([0, 0], dtype=int32))


*Realizamos* la prueba sobre el embebido del texto utlizando make_data iterator, esto toma más tiempo la primera vez dado que debe descargar el embebido pre-establecido

In [9]:
tf.reset_default_graph()

features, label = make_data_iterator("dataFake/fake_train.csv", repeat=1, batch_size=1, shuffle=False)

with tf.Session() as sess:
    tf.tables_initializer().run()
    tf.global_variables_initializer().run() #Linea necesaria para la correcta ejecucion
    f,l = sess.run([features, label])
print(f)
print(l)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
[[-1.55772679e-02  5.67809492e-02  2.41040755e-02 -7.49929249e-02
   4.19605523e-02  7.62734888e-03 -1.69807076e-02 -4.64579612e-02
   2.73969751e-02  3.82841937e-02 -7.03639463e-02  6.23088144e-02
   4.14523110e-03 -1.86321270e-02  7.37708509e-02  6.10872023e-02
  -2.76888888e-02  7.75787011e-02 -2.25997269e-02 -1.91963967e-02
  -7.25625455e-03 -4.41345274e-02  3.20047848e-02  3.24989595e-02
   8.69759843e-02 -3.17422338e-02 -7.62652829e-02 -7.75127634e-02
   9.79043078e-03  9.26115643e-03 -1.29243126e-02  5.63159473e-02
   3.88539094e-03 -8.13603327e-02 -5.52045703e-02  1.02159925e-01
  -3.80212553e-02 -8.15360248e-02 -1.49939507e-02  6.35864660e-02
   3.71673927e-02  5.66056035e-02  6.66051134e-02 -1.62197258e-02
  -4.61176783e-02  5.67898154e-02 -9.

# Usamos un estimador predefinido  (premade estimator)

In [10]:
classifier = tf.estimator.DNNClassifier(hidden_units=[50,10,10], 
                                        activation_fn = tf.nn.relu,
                                        feature_columns=get_features_defs(),                                        
                                        n_classes = len(type_label),
                                        model_dir="models")

tf.logging.set_verbosity(tf.logging.INFO)
!rm -rf models
train_input_fn = lambda: get_dataset("dataFake/fake_train.csv", repeat=2, batch_size=10, shuffle=True)

classifier.train(train_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f454f908610>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'models', '_train_distribute': None, '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized

In [12]:
test_input_fn = lambda: get_dataset("dataFake/fake_test.csv", repeat=1, batch_size=10, shuffle=True)
result = classifier.evaluate(test_input_fn)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-02-13:39:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/model.ckpt-239
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-02-13:39:36
INFO:tensorflow:Saving dict for global step 239: accuracy = 0.42405063, average_loss = 1.3765026, global_step = 239, loss = 13.592963
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 239: models/model.ckpt-239


In [13]:
print(result)

{'average_loss': 1.3765026, 'accuracy': 0.42405063, 'global_step': 239, 'loss': 13.592963}
